# Chapter 1. Statistical Inference
# (통계적 추론)

인구주택총조사처럼 국가에서 작정하고 전수조사를 하는 경우가 아니면 선정한 주제에 맞는 집단들을 모두 조사하고 정보를 모으는 것은 시간과 돈이 너무나 많이 든다. 때문에 보통은 일부 집단을 선택하고 분석해서 모집단이 이런 특성을 가질지 모른다고 추정을 하게 되는데, 일부 집단만 선택해 분석하는 것을 **표본조사(Sampling)**라 하며, 이 때 구할 수 있는 통계량을 바탕으로 모수를 추정하는 것을 **통계적 추론(Statistical inference)**이라 한다.  

## 1.1 Point Estimation
## (점 추정)  
점이라는 단어가 쓰인 것처럼 모수가 특정한 값일 것이라고 추정하는 것을 **점 추정(Point estimation)**이라고 한다.  
수학적으로는 표본평균의 기대값이 모평균과 동일하고, 표본분산의 기대값이 모분산과 동일하다는 것을 이용해 모분산을 구할 수 있다. 좋은 추정량에 대해서는 세 가지 조건이 있는데, 먼저 표본평균의 기대값이 모평균과 동일한 상황이라 가정했을때 추정한 통계량을 **불편추정량(Unbiased estimator)**이라 한다. 또한 불편추정량에 대해 분산을 구했을 때 분산이 가장 적을 때의 통계량을 **효율추정량(Efficient estimator)**이라고 한다. 마지막으로 실제 모수와 차이가 없는 것이나 다름없는 추정량을 **일치추정량(Consistent estimator)**이라 하며, 좋은 점추정은 이 세 경우에 해당하는 추정량을 찾는 것이다. 하지만 실제로는 점 추정을 사실상 안 쓰는데, 일단 코드를 직접 실행하면서 알아보도록 하자.

In [22]:
import numpy as np
import math
import scipy as sp
from scipy import stats as st
from scipy.stats import norm, t, chi2

In [23]:
randints=np.random.randint(1,7500,size=500)  # 모집단 
print("Population :",randints)
sample=np.random.choice(randints,50)   # 표본 집단
print("======"*10)
print("Sample :",sample)
print()
print("Sample Mean :",sample.mean())
print("Sample Variation :",sample.var())
print("Sample Standard Deviation :",sample.std())
print()
print("Unbiased Sample Variation :",sample.var(ddof=1))  
print("Unbiased Sample Standard Deviation :",sample.std(ddof=1))
print()
print("Population Mean :",randints.mean())
print("Population Variation :",randints.var())
print("Population Standard Deviation :",randints.std())

Population : [6450 1965 2867 5507 6882  708 1212 2989 5217 3915 2933 2749 4420 1105
 6711 6905 3185 2341 6872 5313 3906 5530 5554 7431 5643  664 4632 1285
 1551 1234 5204  577 6320 4202 6021    1 2583 5348 1048 5484 5952 4095
 7314 4457  245  447 5157 1136 6996 1977 2139 6920 2171 4400 4563 1000
 3788 2943 2560 4451 6480 1902 4551 7103 5868 1967 4767 7485 3562 6434
  784 1661  568 2230 3111 4204 3178 5043 6999 1157 4865 2561 1630  108
 2270 3359 6585 1585 4779 6977 3681 6708 2663 1683 1406 4842 5635 4547
 3540 4384 5331 7146 5281 4620 5286 3395 6081 3873 5084  382 6431  858
  572 4644 3289 5832 4143 6632 2614 1499  687 7058 4581 3685 1234 2293
  267 1965 4573  733   68 2786 2815 3084  463 6011 2188 7338 7171 4351
 1426 5843   97 2503 3249 6265 3562 1853 2074 2554 6783 2948 6710 6794
 1712 6287  473  687  992  851 3084 3117 2246 1141 4547 5987 5395 5384
 1312 1844 1595 1417  509 6770 5411 5858 5802 7047  244 4026 7104 4165
 1527 7459 3875 4937 6906 1581 2079 2115 3438  182 2941 2459 383

당연하지만 모평균, 모분산, 표본평균, 표본분산을 구할 때에 있어 데이터의 수 자체가 다르기 때문에 표본평균과 표본분산을 통해 모수를 정확하게 구할 수 있을 것이라는 생각은 버리는 것이 좋다. 지금은 random.randint를 만들어서 모집단을 구성했기 때문에 모수를 구할 수 있지만, 현실에선 어지간해서 모수 자체를 구하기가 매우 어렵다. 그렇기 때문에 추정량을 통해 현실적으로 구하기 어려운 모수를 대체하려는 것이다.

## 1.2 Interval Estimation
## (구간 추정)  
앞서 말했듯 점 추정을 통해 표본평균과 표본분산을 구했고, 이들을 모평균과 모분산을 대신하는 값으로 쓰고 싶다. 그러나 수학적으로 그런 이상적인 상황이 꼭 일어날 것이라는 보장이 절대로 없기 때문에, 명확한 값을 가리키는 이 통계량들은 실용성이 떨어진다.  
그래서 대신 생각한 것이 어느 정도 구간에서 표본값들을 계속 찾을 때, 그 중 모수들이 있을 것이라 추정하는 것이다. 이것이 바로 **구간 추정(Interval estimation)**이다. 신뢰도란 같은 구간에서 일정 횟수만큼 표본값을 계속 찾을 때 그 구간 안에 모수가 발견되는 비율을 말하며, 일반적으로 사람들이 납득할 만한 신뢰도를 90%, 95%, 99%로 잡는데, 이를 **신뢰구간(Confidence interval)**이라 한다. 또한 전체에서 신뢰도를 뺐을 때의 비율, 쉽게 말해 재수없게도 모평균의 값이 신뢰구간을 넘어설 확률이 바로 **유의 수준(Significant level) $\alpha$**이다. 각 신뢰구간과 표본의 크기, 모평균을 추정할지 모표준편차를 추정할지에 따라 계산하는 방법이 조금씩 다르다.

먼저 모평균을 추정하는 것부터 시작하자.  
<u>표본의 크기가 30개 이상</u>이면 넓이를 구할 때 신뢰도 상수는 표본정규분포의 확률함수값인 z-값이 되는데, z-값은 신뢰도에 맞는 정규분포함수의 적분값이 되게 하는 확률변수값이 된다. z-값은 신뢰도가 90%면 1.645, 신뢰도가 95%면 1.96, 신뢰도가 99%면 2.58 정도가 되는 것으로 알려져 있지만 여기서는 좀 더 정확한 값을 쓰기 위해 전용 함수를 사용할 것이다. 참고로 표본의 크기가 30개 이상이면 모표준편차를 몰라도 표본표준편차로 대체해 쓸 수 있다.  
$P(\bar{X}-z_{\frac{\alpha}{2}}(\frac{\sigma}{\sqrt{n}}) \le \mu \le 
\bar{X}+z_{\frac{\alpha}{2}}(\frac{\sigma}{\sqrt{n}}))=1-\alpha$  
이때 우리가 구할 신뢰구간은 바로 $[\bar{X}-z_{\frac{\alpha}{2}}(\frac{\sigma}{\sqrt{n}}), \bar{X}+z_{\frac{\alpha}{2}}(\frac{\sigma}{\sqrt{n}})]$이다.
<br/><br/>
<u>표본의 크기가 30개 미만</u>이면 넓이를 구할 때 신뢰도 상수는 t-분포의 확률함수값인 t-값이 되는데, t-값은 자유도가 (표본 크기 -1)일 때 신뢰도에 맞는 정규분포함수의 적분값이 되게 하는 확률변수값이 된다. t-분포는 기본적으로 카이제곱분포에 뿌리를 두고 있기 때문에 t-값은 자유도에 따라 값이 달라진다.  
$P(\bar{X}-t_{\frac{\alpha}{2}}(\frac{s}{\sqrt{n}}) \le \mu \le 
\bar{X}+t_{\frac{\alpha}{2}}(\frac{s}{\sqrt{n}}))=1-\alpha$  
이때 우리가 구할 신뢰구간은 바로 $[\bar{X}-t_{\frac{\alpha}{2}}(\frac{s}{\sqrt{n}}), \bar{X}+t_{\frac{s}{2}}(\frac{s}{\sqrt{n}})]$이다.  
<br/>
참고로 SciPy에서는 확률밀도함수 모듈을 불러서 ppf 함수를 쓸때 계산을 반대로 하므로 주의해야 한다.

In [24]:
# 모표준편차를 알고, 신뢰도가 95%, 표본의 개수가 30개 이상일 때 모평균의 신뢰구간 
sig_level=0.05 # 유의수준 alpha
z_value=norm(0,1).ppf(1-sig_level/2) # alpha에 대한 z-값
a=sample.mean()-z_value*(randints.std()/math.sqrt(len(sample)))
b=sample.mean()+z_value*(randints.std()/math.sqrt(len(sample)))
print("95% Confindence Interval for Population Mean :",[a,b])

95% Confindence Interval for Population Mean : [3355.0360142244303, 4551.403985775569]


In [25]:
# 모표준편차를 모르고, 신뢰도가 95%, 표본의 개수가 30개 이상일 때 모평균의 신뢰구간
sig_level=0.05
z_value=norm(0,1).ppf(1-sig_level/2)
a=sample.mean()-z_value*(sample.std()/math.sqrt(len(sample)))
b=sample.mean()+z_value*(sample.std()/math.sqrt(len(sample)))
print("95% Confindence Interval for Population Mean :",[a,b])

95% Confindence Interval for Population Mean : [3375.9957980936024, 4530.444201906397]


In [26]:
# 모표준편차를 모르고, 신뢰도가 95%, 표본의 개수가 30개 미만일 때 모평균의 신뢰구간
sig_level=0.05
t_value=t.ppf(q=1-sig_level/2,df=len(sample)-1) # 신뢰도가 0.95이고 자유도가 정해졌을 때 t-값
a=sample.mean()-t_value*(sample.std()/math.sqrt(len(sample)))
b=sample.mean()+t_value*(sample.std()/math.sqrt(len(sample)))
print("95% Confindence Interval for Population Mean :",[a,b])

95% Confindence Interval for Population Mean : [3361.384910147006, 4545.055089852994]


모분산에 대한 신뢰구간을 추정하기 위해서는 분산이 퍼져있는 것을 보여주는 카이제곱분포를 써야 한다.  
$P({\frac{(n-1)s^2}{{\chi}^2_{(n-1,\frac{\alpha}{2})}}} \le {\sigma}^2 \le 
{\frac{(n-1)s^2}{{\chi}^2_{(n-1,1-\frac{\alpha}{2})}}})=1-\alpha$  
이때 우리가 구할 신뢰구간은 바로 $[{\frac{(n-1)s^2}{{\chi}^2_{(n-1,\frac{\alpha}{2})}}},{\frac{(n-1)s^2}{{\chi}^2_{(n-1,1-\frac{\alpha}{2})}}}]$이다.  
<br/>
마찬가지로 SciPy에서는 카이제곱함수 모듈을 불러서 ppf 함수를 쓸때 계산을 반대로 하므로 주의해야 한다.

In [27]:
# 신뢰도가 95%일 때 모분산의 신뢰구간
sig_level=0.01
chi_value1=chi2.ppf(q=1-sig_level/2,df=len(sample)-1) # 신뢰도가 0.95이고 자유도가 정해졌을 때 카이제곱값
chi_value2=chi2.ppf(q=sig_level/2,df=len(sample)-1) # 신뢰도가 0.95이고 자유도가 정해졌을 때 카이제곱값
a=(len(sample)-1)*sample.var()/chi_value1
b=(len(sample)-1)*sample.var()/chi_value2
print("95% Confindence Interval for Population Variation :",[a,b])

95% Confindence Interval for Population Variation : [2716324.839255219, 7798351.99093641]


# Chapter 2. Tests of Statistical Hypothesis
# (통계적 가설검정)

이제 모집단의 성질을 몰라도 모평균이 얼추 어디 있는지에 대한 그럴듯한 추정을 할 수 있게 되었다. 그리고 그 추정이 틀릴 수 있다는 것을 순순히 받아들이고 유의수준을 제시함으로서 어느 정도 신뢰성도 얻었다. 지금부터 할 것은 데이터에 따라 어떤 주장을 제시하고, 틀렸을 경우 어떻게 할 지에 대해 판단하는 방법이다.

어떤 정보에 대해 말할 때 이것이 논리적으로 반드시 참이거나 거짓임이 밝혀지진 않았지만 뭔가 그럴듯한 주장을 통계에서는 **가설(Hypothesis)**이라 한다. 그리고 진짜인지는 잘 모르겠지만 일단 믿어보긴 하는데, 막상 증명하기엔 어려운 가설을 **귀무가설(Null hypothesis) $H_0$**, 귀무가설에 완전히 대척점에 있고 증명하는 것이 좀 더 편한 가설을 **대립가설(Alternative hypothesis) $H_1$**이라 한다.  
귀무가설과 대립가설은 논리적으로 상호배타적이기 때문에 어느 한쪽이 틀리면 다른 한쪽은 무조건 맞게 된다. 또한 서로 포괄적인 관계에 있기 때문에 둘 다 틀리거나 둘 다 맞는 상황 자체가 일어나지 않는다. 이런 두가지 특징을 가질 수 있도록 귀무가설과 대립가설을 잘 세워야 할 필요가 있다.

**1종 오류(Type I error)**란 귀무가설이 거짓이라 생각해서 기각했는데 알고보니 귀무가설이 참이었던 것을 의미한다. 위의 구간 추정을 예시로 들면, 실제 표본조사를 통해 얻은 표본평균이 가설로 정한 모평균과 차이가 너무 커서 신뢰 구간을 넘어섰다 생각해 기각했지만 정말로 모평균이 가설로 정한 모평균과 비슷한 상황을 말한다. 구간 추정에서 말한 $\alpha$가 바로 이 1종 오류를 범할 확률로, 재수없게도 표본평균이 극단적인 값이 나오면 벌어지게 된다.  
**2종 오류(Type II error)**란 귀무가설이 참이라 생각해서 채택했는데 알고보니 귀무가설이 거짓이었던 것을 의미한다. 마찬가지로 구간 추정을 예시로 들면, 실제 표본조사를 통해 얻은 표본평균이 가설로 정한 모평균과 차이가 거의 없어 귀무가설을 채택했지만 실제로는 신뢰 구간을 넘어선 영역에 모평균이 있는 상황을 말한다. 이 오류를 범할 확률은 베타($\beta$)라는 값을 따로 쓰며, 이것을 줄이기 위해서는 유의수준 내에서 표본의 크기를 늘려야 한다.  
<u>보통은 2종 오류보다 1종 오류를 더 중요하게 본다.</u> 사람들이 믿는 소문을 귀무가설로, 소문이 거짓이란 썰을 대립가설로 생각하면, 출시한지 얼마 안된 스마트폰이 안 터질 것이라는 소문이 퍼졌을 때, 폰이 안 터질 줄 알았는데 진짜 터지는 것(1종 오류)과, 폰이 터질 거라 생각했는데 알고보니 안터진다(2종 오류)는 것 중에서 어느 쪽이 사회적으로 더 심각하게 받아들일지 생각하면 된다.

<u>**p-값(p-value)**은 유의 수준이 정해졌을 때, 진짜로 1종 오류를 범할 확률을 계산한 값이다.</u> 유의 수준보다 p-값이 작다면 실제로 표본을 계속 뽑았을 때 1종 오류를 범할 확률이 유의 수준보다 낮을 것이라는 말이므로 귀무가설을 기각하고 대립가설을 택할 수 있으며, 유의 수준보다 p-값이 크다면 진짜로 1종 오류를 범할 확률이 예상보다 크므로 귀무가설을 기각할 수 없다.  
유의 수준과 p-값의 가장 큰 차이는, 유의 수준은 우리가 직접 정하는 값이지만 p-값은 계산 결과를 통해 나오는 값이므로 우리가 직접 정할 수 없다.

In [28]:
# 단일 표본에서의 p-값 구하기
from scipy.stats import ttest_1samp
ttest_1samp(sample, popmean = 4127)

Ttest_1sampResult(statistic=-0.5841392514729645, pvalue=0.561806379721258)

표본집단이 하나뿐인 상태에서 예상한 모평균을 정했을 때 검정하는 코드이다. 여기서 모평균이 50이라는 것이 귀무가설, 모평균이 50이 아니라는 것이 귀무가설이다.

In [29]:
# 두 표본이 주어졌을 때 p-값 구하기
from scipy.stats import ttest_rel
ttest_rel(sample, np.random.choice(randints,30))

ValueError: unequal length arrays

이번에는 두 표본이 주어졌을 때 두 표본의 표본평균이 서로 일치하는 경우에 대한 p-값을 구한 것이다. 이때는 두 표본의 표본평균이 서로 일치한다는 것이 귀무가설, 이 경우도 마찬가지로, 아니라는 것이 대립가설이다.  

In [30]:
# 두 표본의 상관관계에 대한 p-값 구하기
import pandas as pd
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(sample, np.random.choice(randints,30)))[1]

ValueError: arrays must all be same length

모수의 값뿐만 아니라 두 표본의 상관관계에 대해 검정하는 것도 가능하다. 여기서는 두 표본집단에 대한 상관관계가 없다는 것이 귀무가설, 상관관계가 있다는 것이 대립가설이다.  
chi2_contingency 배열의 2번째 성분이 p-값이므로 인덱스를 1로 해서 코드를 실행하면 p-값만 볼 수 있다.

그 외에도 여러 가설 검정 방법이 있으니 검색해보자. 사실 여기선 표본의 크기 자체가 작기 때문에 실제 모평균과 표본평균의 차이가 클 수밖에 없으며, p-값이 유의 수준보다 큰 경우가 계속 나오는 것도 이 때문이다. 표본의 크기를 더 늘린다면 반대의 경우가 더 많이 나올수는 있지만, 반드시 그럴 것이라는 보장은 없다.

# 마무리

이번 시간에는 통계의 아주 기본적인 개념인 기대값, 확률함수, 표본통계량에 대한 설명은 건너뛰고 바로 모수 추정과 가설 검정으로 나갔기 때문에 이들을 잘 모른다면 따로 공부할 필요가 있다. 또한 다변수 확률에 대한 검정을 하지 못했기 때문에 기대값을 이용한 공분산과 상관관계의 정의를 먼저 공부한 다음 이 내용을 기반으로 다변수 모수 추정 및 검정에 대해 알아보면 이해가 좀 더 수월할 것이다.  
  
통계를 배울 때 확실하게 알아야 하는 것은 통계에서 쓰이는 확률은 절대 만능이 아니며, 처음부터 잘못된 판단이 있을 수 있다는 점을 감안하고 쓰이는 값이라는 것이다. 다만 잘못된 판단을 저지를 확률을 어떻게든 낮추기 위해 수학적 논리를 사용하고 특수한 기준을 세운 덕분에 비로소 다른 사람들이 납득할 수 있게 된 것뿐이다. 다른 사람들에게 본인이 찾아낸 확률과 모수를 바탕으로 본인의 주장에 설득력을 높이고 싶다면 그만큼 통계량에 대한 충분한 이해가 있어야 가능하다. 좀 더 깊게 공부하고 싶다면 따로 수리통계 교재를 찾아 증명과정을 상세하게 공부하길 권장한다.  
  
대부분의 사람은 확률에 대해 크고 작은 선입견과 잘못된 지식을 갖고 있다. 나 또한 이 자료를 정리하면서 나조차 잘못 알고 있던 정의에 대해 바로잡는 계기가 되었다. 이 파일이 모수 추정과 가설 검정에 대해 크고 작은 도움을 주길 바라며, 여러분들은 앞서 말한 과오들을 저지르지 않았으면 한다.
<br/><br/><br/>
-끝-